In [31]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType


In [32]:
val Spark = SparkSession
 .builder
 .appName("Google Apps Exercise")
 .getOrCreate()

Spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@48363047


In [33]:
val csvFile= "googleplaystore.csv"

val Dframe = Spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)

csvFile: String = googleplaystore.csv
Dframe: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 11 more fields]


In [34]:
Dframe.filter(Dframe("App") === "Infinite Painter").show()

+----------------+--------------+------+-------+----+----------+----+-----+--------------+------------+-------------+-----------+-----------+
|             App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|      Genres| Last Updated|Current Ver|Android Ver|
+----------------+--------------+------+-------+----+----------+----+-----+--------------+------------+-------------+-----------+-----------+
|Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 29M|1,000,000+|Free|    0|      Everyone|Art & Design|June 14, 2018|   6.1.61.1| 4.2 and up|
+----------------+--------------+------+-------+----+----------+----+-----+--------------+------------+-------------+-----------+-----------+



In [35]:
Dframe.createOrReplaceTempView("Google_Apps_tmp")

In [36]:
Spark.sql("SELECT count(App) as Total FROM Google_Apps_tmp WHERE App='Infinite Painter'").show()

+-----+
|Total|
+-----+
|    1|
+-----+



EJERCICIO 1

In [37]:
val Dframe_updated = Dframe.filter(not($"App".equalTo("Infinite Painter")))
Dframe_updated.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

Dframe_updated: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [App: string, Category: string ... 11 more fields]


EJERCICIO 2

In [38]:
val replace_NaN = Dframe.withColumn("Rating", when(col("Rating") === "NaN", "0").otherwise(col("Rating")))
replace_NaN.filter(replace_NaN("Rating") === "NaN").count()

replace_NaN: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 11 more fields]
res28: Long = 0


EJERCICIO 3

In [19]:
val replace_NaN_Type = replace_NaN.withColumn("Type", when(col("Type") === "NaN", "Unknow").otherwise(col("Type")))
replace_NaN_Type.filter(replace_NaN_Type("Type") === "Unknow").count()

replace_NaN_Type: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 11 more fields]
res12: Long = 1


EJERCICIO 5

In [30]:
val int_df = replace_NaN_Type.withColumn("Installs",col("Installs").cast(IntegerType))


val Rating_Categorical_dframe = replace_NaN_Type.withColumn("Rating_Categorical", 
      expr("case when Rating < '2' then 'Baja'" + 
           "when Rating >= '2' AND Rating<'3' then 'Media' " +
           "when Rating >= '3' AND Rating<'4' then 'Alta'" +
           "when Rating >= '4' then 'Muy Alta'" +
           "else 'Unknown' end"))

Rating_Categorical_dframe.filter(Rating_Categorical_dframe("Rating_Categorical")==="Baja")
Rating_Categorical_dframe.groupBy("Rating_Categorical").agg(count("Rating_Categorical").alias("Número de valoraciones")).show()

+------------------+----------------------+
|Rating_Categorical|Número de valoraciones|
+------------------+----------------------+
|              Alta|                  1711|
|          Muy Alta|                  7367|
|             Media|                   231|
|              Baja|                  1532|
+------------------+----------------------+



int_df: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 11 more fields]
Rating_Categorical_dframe: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 12 more fields]


EJERCICIO 6 A

In [42]:
val dframe_limpio = Rating_Categorical_dframe
dframe_limpio.select("App","Rating").filter(dframe_limpio("Rating")>4.5).show()

+--------------------+------+
|                 App|Rating|
+--------------------+------+
|U Launcher Lite –...|   4.7|
|Kids Paint Free -...|   4.7|
|Mandala Coloring ...|   4.6|
|Photo Designer - ...|   4.7|
|        ibis Paint X|   4.6|
|Superheroes Wallp...|   4.7|
|HD Mickey Minnie ...|   4.7|
|Harley Quinn wall...|   4.8|
|Colorfit - Drawin...|   4.7|
|     I Creative Idea|   4.7|
|UNICORN - Color B...|   4.7|
|PIP Camera - PIP ...|   4.7|
|Canva: Poster, ba...|   4.7|
|Install images wi...|   4.6|
|Used Cars and Tru...|   4.6|
|Tickets SDA 2018 ...|   4.9|
|        CityBus Lviv|   4.6|
|CDL Practice Test...|   4.9|
|Free VIN Report f...|   4.6|
|DMV Permit Practi...|   4.9|
+--------------------+------+
only showing top 20 rows



dframe_limpio: org.apache.spark.sql.DataFrame = [App: string, Category: string ... 12 more fields]


EJERCICIO 6 B

In [47]:
dframe_limpio.select("App","Rating_Categorical","Type").filter(dframe_limpio("Rating_Categorical")==="Muy Alta").filter(dframe_limpio("Type")==="Free").count()

res36: Long = 6840


EJERCICIO 6C

In [72]:
dframe_limpio.select("App","Price").filter(dframe_limpio("Price")<"13").filter(dframe_limpio("Price")!=="0").show()

+--------------------+-----+
|                 App|Price|
+--------------------+-----+
|TurboScan: scan d...|$4.99|
|Tiny Scanner Pro:...|$4.99|
|TurboScan: scan d...|$4.99|
|Tiny Scanner Pro:...|$4.99|
|  Puffin Browser Pro|$3.99|
|Moco+ - Chat, Mee...|$3.99|
|          Calculator|$6.99|
|   Truth or Dare Pro|$1.49|
|Private Dating, H...|$2.99|
|Ad Blocker for SayHi|$3.99|
|AMBW Dating App: ...|$7.99|
|Moco+ - Chat, Mee...|$3.99|
| Sago Mini Hat Maker|$3.99|
|Fuzzy Numbers: Pr...|$5.99|
|     Toca Life: City|$3.99|
| Toca Life: Hospital|$3.99|
|      My Talking Pet|$4.99|
|      Meme Generator|$2.99|
|My CookBook Pro (...|$3.49|
|Paprika Recipe Ma...|$4.99|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df.sample(0.1).collect()